In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# 데이터 파일 로드
file_path = "pet_food_customer_orders.csv"
df = pd.read_csv(file_path)

# 필요한 열 선택 및 결측치 처리 (이전과 동일)
df_model = df[[
    'pet_life_stage_at_order',  # 노견 여부
    'pet_allergen_list',        # 알레르기 여부
    'total_web_sessions',       # 총 웹 세션
    'total_minutes_on_website'  # 총 웹사이트 체류 시간
]].copy()

df_model['pet_allergen_list'].fillna('None', inplace=True)
df_model['pet_allergen_list'].replace('Null & Default', 'None', inplace=True)
df_model[['total_web_sessions', 'total_minutes_on_website']] = \
    df_model[['total_web_sessions', 'total_minutes_on_website']].fillna(0)

# --- 새로운 Y 변수 정의 ---
# 🌟 Y = 1 (타겟 그룹): 노견(mature)이거나 알레르기가 있는 경우
is_mature = (df_model['pet_life_stage_at_order'] == 'mature')
has_allergy = (df_model['pet_allergen_list'] != 'None')
df_model['Y'] = (is_mature | has_allergy).astype(int)

# 🌟 Y = 0 (대조 그룹): 노견도 아니고 알레르기도 없는 '건강한' 경우
# Y=0을 명시적으로 정의할 필요 없이, Y=1이 아닌 모든 데이터가 Y=0이 됩니다.
# 즉, (not is_mature) & (not has_allergy) 인 경우가 Y=0입니다.

print(f"✅ 전체 데이터 중 Y=1 (노견 또는 알레르기) 그룹 개수: {df_model['Y'].sum()}")
print(f"✅ 전체 데이터 중 Y=0 (건강한) 그룹 개수: {len(df_model) - df_model['Y'].sum()}")

✅ 전체 데이터 중 Y=1 (노견 또는 알레르기) 그룹 개수: 32737
✅ 전체 데이터 중 Y=0 (건강한) 그룹 개수: 16305


C:\Users\user\AppData\Local\Temp\ipykernel_14820\10865853.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_model['pet_allergen_list'].fillna('None', inplace=True)


In [2]:
# 1. Target Group (Y=1)과 Control Group (Y=0) 데이터 분리
df_target = df_model[df_model['Y'] == 1]
df_control = df_model[df_model['Y'] == 0]

# 2. 소수 그룹의 크기를 기준으로 설정
if len(df_target) <= len(df_control):
    n_minority = len(df_target)
    df_majority = df_control
    df_minority = df_target
else:
    n_minority = len(df_control)
    df_majority = df_target
    df_minority = df_control

# 3. 다수 그룹을 소수 그룹의 크기에 맞게 랜덤 샘플링 (1:1 분포)
# random_state를 설정하여 결과의 재현성을 확보
df_majority_sampled = df_majority.sample(n=n_minority, random_state=42)

# 4. 샘플링된 두 그룹 데이터를 합치기
df_balanced = pd.concat([df_minority, df_majority_sampled])

print("\n" + "-"*50)
print(f"✅ 소수 그룹 크기: {n_minority}")
print(f"✅ 최종 균형 잡힌 데이터 크기 (1:1 비율): {len(df_balanced)}")
print("-" * 50)

# 독립 변수(X)와 종속 변수(Y) 정의
X = df_balanced[['total_web_sessions', 'total_minutes_on_website']]
Y = df_balanced['Y']


--------------------------------------------------
✅ 소수 그룹 크기: 16305
✅ 최종 균형 잡힌 데이터 크기 (1:1 비율): 32610
--------------------------------------------------


In [4]:
# 상수항(Intercept) 추가
X = sm.add_constant(X) 

# 로지스틱 회귀 모델 적합 (fit)
logit_model = sm.Logit(Y, X)
result = logit_model.fit()

# 결과 출력
print("### 📊 로지스틱 회귀분석 결과 (Statsmodels Logit) ###")
print(result.summary())

# 승산비(Odds Ratio) 해석을 위한 지수(Exponential) 변환
odds_ratios = np.exp(result.params)
print("\n### ✨ Odds Ratio (승산비) ###")
print(f"상수항 (Intercept) 승산비: {odds_ratios['const']:.4f}")
print(f"총 웹 세션 승산비: {odds_ratios['total_web_sessions']:.4f}")
print(f"총 체류 시간 승산비: {odds_ratios['total_minutes_on_website']:.4f}")
print("-" * 50)

Optimization terminated successfully.
         Current function value: 0.690489
         Iterations 4
### 📊 로지스틱 회귀분석 결과 (Statsmodels Logit) ###
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                32610
Model:                          Logit   Df Residuals:                    32607
Method:                           MLE   Df Model:                            2
Date:                Tue, 11 Nov 2025   Pseudo R-squ.:                0.003835
Time:                        14:26:24   Log-Likelihood:                -22517.
converged:                       True   LL-Null:                       -22604.
Covariance Type:            nonrobust   LLR p-value:                 2.234e-38
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                       -0.1104  

In [11]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# 데이터 파일 로드
file_path = "pet_food_customer_orders.csv"
df = pd.read_csv(file_path)

# --- 필요한 열 선택 및 결측치 처리 ---
df_model = df[[
    'pet_life_stage_at_order',  # 노견 여부
    'pet_allergen_list',        # 알레르기 여부
    'total_web_sessions',       # 웹 세션
    'total_minutes_on_website', # 체류 시간
    'pet_food_tier'             # ⭐ 신규 변수 추가
]].copy()

# 결측치 처리 (이전과 동일)
df_model['pet_allergen_list'].fillna('None', inplace=True)
df_model['pet_allergen_list'].replace('Null & Default', 'None', inplace=True)
df_model[['total_web_sessions', 'total_minutes_on_website']] = \
    df_model[['total_web_sessions', 'total_minutes_on_website']].fillna(0)

# ⭐ Y 변수 정의 (노견 또는 알레르기 = 1)
is_mature = (df_model['pet_life_stage_at_order'] == 'mature')
has_allergy = (df_model['pet_allergen_list'] != 'None')
df_model['Y'] = (is_mature | has_allergy).astype(int)

# --- 💡 변수 변환 및 생성 (Feature Engineering) ---
# 1. 로그 변환 (로그(1 + X) 사용)
df_model['log_sessions'] = np.log1p(df_model['total_web_sessions'])
df_model['log_minutes'] = np.log1p(df_model['total_minutes_on_website'])

# 2. 세션당 평균 체류 시간 생성
# total_web_sessions가 0인 경우 0으로 처리하여 나누기 오류 방지
df_model['avg_min_per_session'] = np.where(
    df_model['total_web_sessions'] > 0,
    df_model['total_minutes_on_website'] / df_model['total_web_sessions'],
    0
)

# 3. 범주형 변수 더미화 (pet_food_tier)
# 'superpremium', 'premium', 'mid' 등으로 구성되어 있으며,
# Get Dummies를 통해 자동으로 더미 변수 생성 및 원본 변수 제거
df_dummies = pd.get_dummies(df_model['pet_food_tier'], prefix='tier', drop_first=True)
df_model = pd.concat([df_model, df_dummies], axis=1)

# print(df_model.head()) # 변수 생성 확인용

C:\Users\user\AppData\Local\Temp\ipykernel_14820\3897191035.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_model['pet_allergen_list'].fillna('None', inplace=True)


In [12]:
# --- 1:1 분포를 위한 언더샘플링 (이전과 동일) ---
df_target = df_model[df_model['Y'] == 1]
df_control = df_model[df_model['Y'] == 0]

# 소수 그룹의 크기를 기준으로 샘플링
if len(df_target) <= len(df_control):
    n_minority = len(df_target)
    df_majority = df_control
    df_minority = df_target
else:
    n_minority = len(df_control)
    df_majority = df_target
    df_minority = df_control

df_majority_sampled = df_majority.sample(n=n_minority, random_state=42)
df_balanced = pd.concat([df_minority, df_majority_sampled])

print(f"✅ 균형 잡힌 최종 데이터 크기 (1:1 비율): {len(df_balanced)}")
print("-" * 50)

# --- 💡 독립 변수(X) 재정의 ---
# 변환된 웹 활동 변수와 더미 변수를 포함
X_cols = [
    'log_sessions', 
    'log_minutes', 
    'avg_min_per_session'
] + [col for col in df_balanced.columns if col.startswith('tier_')]

X = df_balanced[X_cols]
Y = df_balanced['Y']

# 상수항(Intercept) 추가 및 로지스틱 회귀 모델 적합
X = sm.add_constant(X) 
X = X.astype(float)
logit_model = sm.Logit(Y, X)
result = logit_model.fit(disp=False) # disp=False: 학습 과정 메시지 생략

# 결과 출력
print("### 📊 변환/추가 변수 적용 로지스틱 회귀분석 결과 ###")
print(result.summary())

# 승산비(Odds Ratio) 해석
odds_ratios = np.exp(result.params)
print("\n### ✨ Odds Ratio (승산비) ###")
print(odds_ratios)
print("-" * 50)

✅ 균형 잡힌 최종 데이터 크기 (1:1 비율): 32610
--------------------------------------------------
### 📊 변환/추가 변수 적용 로지스틱 회귀분석 결과 ###
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                32610
Model:                          Logit   Df Residuals:                    32604
Method:                           MLE   Df Model:                            5
Date:                Tue, 11 Nov 2025   Pseudo R-squ.:                0.003115
Time:                        14:33:25   Log-Likelihood:                -22533.
converged:                       True   LL-Null:                       -22604.
Covariance Type:            nonrobust   LLR p-value:                 1.201e-28
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                  -0.2654      0.036     -7.423      0.000      -0.

In [14]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report

# 데이터 파일 로드
file_path = "pet_food_customer_orders.csv"
df = pd.read_csv(file_path)

# --- 1. 이전 단계에서 정의된 Y, 변환된 변수, 더미 변수 생성 코드 재실행 ---

df_model = df[[
    'pet_life_stage_at_order', 'pet_allergen_list', 'total_web_sessions',
    'total_minutes_on_website', 'pet_food_tier',
    'pet_breed_size', 'neutered', 'pet_health_issue_list'
]].copy()

# 결측치 및 'Null & Default' 처리
df_model['pet_food_tier'].fillna('missing', inplace=True)
df_model['pet_food_tier'].replace('Null & Default', 'missing', inplace=True)
df_model['pet_allergen_list'].fillna('None', inplace=True)
df_model['pet_allergen_list'].replace('Null & Default', 'None', inplace=True)
df_model[['total_web_sessions', 'total_minutes_on_website']] = \
    df_model[['total_web_sessions', 'total_minutes_on_website']].fillna(0)

# Y 변수 정의 (노견 또는 알레르기 = 1)
is_mature = (df_model['pet_life_stage_at_order'] == 'mature')
has_allergy = (df_model['pet_allergen_list'] != 'None')
df_model['Y'] = (is_mature | has_allergy).astype(int)

# 변수 변환 및 더미 변수 생성
df_model['log_sessions'] = np.log1p(df_model['total_web_sessions'])

df_dummies_tier = pd.get_dummies(df_model['pet_food_tier'], prefix='tier', drop_first=True)
df_dummies_size = pd.get_dummies(df_model['pet_breed_size'], prefix='size', drop_first=True)

df_model['is_neutered'] = (df_model['neutered'].astype(str).str.upper() == 'TRUE').astype(int)
df_model['has_other_health_issue'] = df_model['pet_health_issue_list'].apply(
    lambda x: 1 if pd.notna(x) and str(x) not in ['None', 'Null & Default', ''] else 0
)

df_model = pd.concat([df_model, df_dummies_tier, df_dummies_size], axis=1)

# --- 2. 1:1 분포를 위한 언더샘플링 (이전과 동일) ---
df_target = df_model[df_model['Y'] == 1]
df_control = df_model[df_model['Y'] == 0]

if len(df_target) <= len(df_control):
    n_minority = len(df_target)
    df_majority = df_control
    df_minority = df_target
else:
    n_minority = len(df_control)
    df_majority = df_target
    df_minority = df_control

df_majority_sampled = df_majority.sample(n=n_minority, random_state=42)
df_balanced = pd.concat([df_minority, df_majority_sampled])

# --- 3. XGBoost를 위한 독립 변수(X) 최종 정의 ---
X_cols_final = [
    'log_sessions', 'is_neutered', 'has_other_health_issue',
    'tier_premium', 'tier_superpremium', 'tier_missing', 
    'size_medium', 'size_small', 'size_Null & Default', 'size_missing'
]

X_cols_final = [col for col in X_cols_final if col in df_balanced.columns]

X = df_balanced[X_cols_final].astype(float)
Y = df_balanced['Y']

# 학습 및 테스트 데이터 분리 (모델 성능 검증을 위해 필수)
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42, stratify=Y
)

# 4. XGBoost 모델 학습
xgb_model = xgb.XGBClassifier(
    objective='binary:logistic',
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42,
    n_estimators=100, # 트리 개수
    max_depth=3       # 트리 깊이 (과적합 방지)
)

print("✅ XGBoost 모델 학습 중...")
xgb_model.fit(X_train, Y_train)

C:\Users\user\AppData\Local\Temp\ipykernel_14820\840882787.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_model['pet_food_tier'].fillna('missing', inplace=True)
C:\Users\user\AppData\Local\Temp\ipykernel_14820\840882787.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.



✅ XGBoost 모델 학습 중...


c:\Users\user\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [14:36:52] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, ...)

In [17]:
# 예측 수행
Y_pred = xgb_model.predict(X_test)
Y_proba = xgb_model.predict_proba(X_test)[:, 1]

# 평가 지표 출력
print(f"정확도 (Accuracy): {accuracy_score(Y_test, Y_pred):.4f}")
print(f"ROC-AUC 점수: {roc_auc_score(Y_test, Y_proba):.4f}")
print("\n분류 보고서 (Classification Report):")
print(classification_report(Y_test, Y_pred))

# 변수 중요도 (Feature Importance) 확인
importance = pd.Series(xgb_model.feature_importances_, index=X.columns)
importance = importance.sort_values(ascending=False)

print("\n### 🔑 변수 중요도 순위 (Feature Importance) ###")
print(importance)
print("-" * 50)

정확도 (Accuracy): 0.6352
ROC-AUC 점수: 0.6714

분류 보고서 (Classification Report):
              precision    recall  f1-score   support

           0       0.64      0.60      0.62      3261
           1       0.63      0.67      0.65      3261

    accuracy                           0.64      6522
   macro avg       0.64      0.64      0.63      6522
weighted avg       0.64      0.64      0.63      6522


### 🔑 변수 중요도 순위 (Feature Importance) ###
is_neutered               0.792006
has_other_health_issue    0.109410
tier_premium              0.032367
tier_superpremium         0.020638
size_small                0.018637
log_sessions              0.015226
size_medium               0.011717
dtype: float32
--------------------------------------------------


In [18]:
import pandas as pd
import numpy as np
from scipy import stats # T-검정을 위한 라이브러리

# 데이터 파일 로드
file_path = "pet_food_customer_orders.csv"
df = pd.read_csv(file_path)

# --- 타겟 변수 및 변수 준비 ---
df_analysis = df.copy()

# 결측치 처리 (웹 활동 변수와 알레르기/노령견 변수)
df_analysis['pet_allergen_list'].fillna('None', inplace=True)
df_analysis['pet_allergen_list'].replace('Null & Default', 'None', inplace=True)
df_analysis[['total_web_sessions', 'total_minutes_on_website']] = \
    df_analysis[['total_web_sessions', 'total_minutes_on_website']].fillna(0)

# ⭐ Y 변수 정의 ⭐
# Target Group (Y=1): 노령견(mature)이거나 알레르기가 있는 경우
is_mature = (df_analysis['pet_life_stage_at_order'] == 'mature')
has_allergy = (df_analysis['pet_allergen_list'] != 'None')
df_analysis['Target_Group'] = (is_mature | has_allergy).astype(int)

# 1. 노령견/알레르기 그룹 (Y=1)
group_target = df_analysis[df_analysis['Target_Group'] == 1]
# 2. 일반 그룹 (Y=0)
group_control = df_analysis[df_analysis['Target_Group'] == 0]

print(f"✅ Target Group (노령견/알레르기) 수: {len(group_target)}")
print(f"✅ Control Group (일반) 수: {len(group_control)}")
print("\n" + "-"*50)

✅ Target Group (노령견/알레르기) 수: 32737
✅ Control Group (일반) 수: 16305

--------------------------------------------------


C:\Users\user\AppData\Local\Temp\ipykernel_14820\397729491.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_analysis['pet_allergen_list'].fillna('None', inplace=True)


In [19]:
print("### 📊 그룹별 웹 활동 기술통계량 ###")

# 총 웹 세션
stats_sessions = df_analysis.groupby('Target_Group')['total_web_sessions'].agg(
    ['count', 'mean', 'median', 'std', 'min', 'max']
)
print("\n[총 웹 세션 (total_web_sessions)]")
print(stats_sessions.rename(index={0: '일반 그룹 (Y=0)', 1: '타겟 그룹 (Y=1)'}))

# 총 체류 시간
stats_minutes = df_analysis.groupby('Target_Group')['total_minutes_on_website'].agg(
    ['count', 'mean', 'median', 'std', 'min', 'max']
)
print("\n[총 웹 체류 시간 (total_minutes_on_website)]")
print(stats_minutes.rename(index={0: '일반 그룹 (Y=0)', 1: '타겟 그룹 (Y=1)'}))
print("-" * 50)

### 📊 그룹별 웹 활동 기술통계량 ###

[총 웹 세션 (total_web_sessions)]
              count      mean  median        std  min  max
Target_Group                                              
일반 그룹 (Y=0)   16305  7.192150     4.0   8.923336    0  108
타겟 그룹 (Y=1)   32737  8.345878     5.0  10.089429    0  124

[총 웹 체류 시간 (total_minutes_on_website)]
              count        mean  median         std  min    max
Target_Group                                                   
일반 그룹 (Y=0)   16305  372.165164    52.0  798.097090    0  17318
타겟 그룹 (Y=1)   32737  389.271222    62.0  833.859941    0  23734
--------------------------------------------------


In [20]:
print("### 🔬 독립표본 T-검정 결과 (평균 차이) ###")

# A. 총 웹 세션 T-검정
t_stat_sessions, p_val_sessions = stats.ttest_ind(
    group_target['total_web_sessions'], 
    group_control['total_web_sessions'], 
    equal_var=False # Welch's T-test (등분산성 가정하지 않음)
)

print("\n[총 웹 세션 평균 차이]")
print(f"T-statistic: {t_stat_sessions:.4f}")
print(f"P-value: {p_val_sessions:.5f}")
if p_val_sessions < 0.05:
    print("➡️ 해석: P-value가 0.05 미만이므로, 두 그룹의 웹 세션 평균은 통계적으로 유의미하게 다릅니다.")
else:
    print("➡️ 해석: P-value가 0.05 이상이므로, 두 그룹의 웹 세션 평균 차이는 통계적으로 유의미하지 않습니다.")


# B. 총 체류 시간 T-검정
t_stat_minutes, p_val_minutes = stats.ttest_ind(
    group_target['total_minutes_on_website'], 
    group_control['total_minutes_on_website'], 
    equal_var=False
)

print("\n[총 웹 체류 시간 평균 차이]")
print(f"T-statistic: {t_stat_minutes:.4f}")
print(f"P-value: {p_val_minutes:.5f}")
if p_val_minutes < 0.05:
    print("➡️ 해석: P-value가 0.05 미만이므로, 두 그룹의 웹 체류 시간 평균은 통계적으로 유의미하게 다릅니다.")
else:
    print("➡️ 해석: P-value가 0.05 이상이므로, 두 그룹의 웹 체류 시간 평균 차이는 통계적으로 유의미하지 않습니다.")
print("-" * 50)

### 🔬 독립표본 T-검정 결과 (평균 차이) ###

[총 웹 세션 평균 차이]
T-statistic: 12.9047
P-value: 0.00000
➡️ 해석: P-value가 0.05 미만이므로, 두 그룹의 웹 세션 평균은 통계적으로 유의미하게 다릅니다.

[총 웹 체류 시간 평균 차이]
T-statistic: 2.2028
P-value: 0.02762
➡️ 해석: P-value가 0.05 미만이므로, 두 그룹의 웹 체류 시간 평균은 통계적으로 유의미하게 다릅니다.
--------------------------------------------------


In [21]:
import pandas as pd
import numpy as np

# 데이터 파일 로드
file_path = "pet_food_customer_orders.csv"
df = pd.read_csv(file_path)

# --- 타겟 변수 및 변수 준비 ---
df_analysis = df.copy()

# 결측치 처리 (웹 활동 변수와 알레르기/노령견 변수)
df_analysis['pet_allergen_list'].fillna('None', inplace=True)
df_analysis['pet_allergen_list'].replace('Null & Default', 'None', inplace=True)
df_analysis[['total_web_sessions', 'total_minutes_on_website']] = \
    df_analysis[['total_web_sessions', 'total_minutes_on_website']].fillna(0)

# ⭐ 3가지 그룹 정의 ⭐
# 1. 노령견 그룹 (Y=1): mature 이면서 알레르기가 없는 경우
is_mature = (df_analysis['pet_life_stage_at_order'] == 'mature')
has_allergy = (df_analysis['pet_allergen_list'] != 'None')

# 'Group_Type' 컬럼 생성
df_analysis['Group_Type'] = np.select(
    [
        is_mature & ~has_allergy,   # 조건 1: 노령견(mature)이면서 알레르기가 없는 경우
        has_allergy & ~is_mature,   # 조건 2: 알레르기가 있으면서 노령견이 아닌 경우
        is_mature & has_allergy,    # 조건 3: 노령견이면서 알레르기가 있는 경우
    ],
    [
        'A. 노령견 그룹', 
        'B. 알레르기 그룹', 
        'C. 복합 그룹 (노령견 & 알레르기)'
    ],
    default='D. 일반 그룹 (모두 해당 없음)' # 기본값
)

# 분석에서 '복합 그룹'은 제외하고, 순수한 3개 그룹을 비교
# 만약 '복합 그룹'까지 포함하여 분석하려면, 위의 np.select 조건을 수정해야 합니다.
# 여기서는 요청하신 '노령견', '알레르기', '일반' 그룹을 명확히 비교하기 위해
# 노령견이면서 알레르기인 경우는 제외하고 순수 그룹을 정의하겠습니다.

# 💡 재정의: 겹치는 그룹을 분류합니다.
def categorize_group(row):
    is_mature_ = (row['pet_life_stage_at_order'] == 'mature')
    has_allergy_ = (row['pet_allergen_list'] != 'None')
    
    if is_mature_ and has_allergy_:
        return 'C. 복합 그룹' # 노령견이면서 알레르기 (분석 제외 또는 별도 고려)
    elif is_mature_:
        return 'A. 노령견 그룹'
    elif has_allergy_:
        return 'B. 알레르기 그룹'
    else:
        return 'D. 일반 그룹'

df_analysis['Final_Group'] = df_analysis.apply(categorize_group, axis=1)

# 분석에 사용할 3개 그룹 선택
target_groups = ['A. 노령견 그룹', 'B. 알레르기 그룹', 'D. 일반 그룹']
df_filtered = df_analysis[df_analysis['Final_Group'].isin(target_groups)].copy()

print(f"✅ 최종 분석 대상 그룹별 개수:")
print(df_filtered['Final_Group'].value_counts().sort_index())
print("\n" + "-"*50)

C:\Users\user\AppData\Local\Temp\ipykernel_14820\702703663.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_analysis['pet_allergen_list'].fillna('None', inplace=True)


✅ 최종 분석 대상 그룹별 개수:
Final_Group
A. 노령견 그룹     21979
B. 알레르기 그룹     3604
D. 일반 그룹      16305
Name: count, dtype: int64

--------------------------------------------------


In [22]:
print("### 📊 그룹별 웹 활동 기술통계량 ###")

# 총 웹 세션
stats_sessions = df_filtered.groupby('Final_Group')['total_web_sessions'].agg(
    ['count', 'mean', 'median', 'std', 'min', 'max']
).sort_index()

print("\n[총 웹 세션 (total_web_sessions)]")
print(stats_sessions)

# 총 체류 시간
stats_minutes = df_filtered.groupby('Final_Group')['total_minutes_on_website'].agg(
    ['count', 'mean', 'median', 'std', 'min', 'max']
).sort_index()

print("\n[총 웹 체류 시간 (total_minutes_on_website)]")
print(stats_minutes)
print("-" * 50)

### 📊 그룹별 웹 활동 기술통계량 ###

[총 웹 세션 (total_web_sessions)]
             count      mean  median       std  min  max
Final_Group                                             
A. 노령견 그룹    21979  8.082488     5.0  9.976101    0  124
B. 알레르기 그룹    3604  8.341565     5.0  9.785098    0   73
D. 일반 그룹     16305  7.192150     4.0  8.923336    0  108

[총 웹 체류 시간 (total_minutes_on_website)]
             count        mean  median         std  min    max
Final_Group                                                   
A. 노령견 그룹    21979  374.211065    55.0  849.145445    0  23734
B. 알레르기 그룹    3604  413.411487    70.0  822.256032    0  17365
D. 일반 그룹     16305  372.165164    52.0  798.097090    0  17318
--------------------------------------------------


In [23]:
import pandas as pd
import numpy as np

# 데이터 파일 로드
file_path = "pet_food_customer_orders.csv"
df = pd.read_csv(file_path)

# --- 타겟 변수 및 변수 준비 ---
df_analysis = df.copy()

# 결측치 처리
df_analysis['pet_allergen_list'].fillna('None', inplace=True)
df_analysis['pet_allergen_list'].replace('Null & Default', 'None', inplace=True)
# pet_food_tier의 결측치 및 Null 값은 'missing'으로 처리하여 그룹에 포함
df_analysis['pet_food_tier'].fillna('missing', inplace=True)
df_analysis['pet_food_tier'].replace('Null & Default', 'missing', inplace=True)

# ⭐ 3가지 그룹 정의 (노령견이면서 알레르기인 복합 그룹은 제외) ⭐
def categorize_group(row):
    is_mature_ = (row['pet_life_stage_at_order'] == 'mature')
    has_allergy_ = (row['pet_allergen_list'] != 'None')
    
    if is_mature_ and has_allergy_:
        return 'C. 복합 그룹 (제외)' 
    elif is_mature_:
        return 'A. 노령견 그룹'
    elif has_allergy_:
        return 'B. 알레르기 그룹'
    else:
        return 'D. 일반 그룹'

df_analysis['Final_Group'] = df_analysis.apply(categorize_group, axis=1)

# 복합 그룹을 제외한 3개 그룹만 필터링
target_groups = ['A. 노령견 그룹', 'B. 알레르기 그룹', 'D. 일반 그룹']
df_filtered = df_analysis[df_analysis['Final_Group'].isin(target_groups)].copy()

print(f"✅ 최종 분석 대상 그룹별 개수:")
print(df_filtered['Final_Group'].value_counts().sort_index())
print("\n" + "-"*50)

C:\Users\user\AppData\Local\Temp\ipykernel_14820\549736833.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_analysis['pet_allergen_list'].fillna('None', inplace=True)
C:\Users\user\AppData\Local\Temp\ipykernel_14820\549736833.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a cop

✅ 최종 분석 대상 그룹별 개수:
Final_Group
A. 노령견 그룹     21979
B. 알레르기 그룹     3604
D. 일반 그룹      16305
Name: count, dtype: int64

--------------------------------------------------


In [24]:
print("### 📊 그룹별 사료 등급 선호도 (빈도) ###")

# 1. 빈도표 (Count)
tier_counts = pd.crosstab(
    df_filtered['Final_Group'], 
    df_filtered['pet_food_tier'], 
    margins=True # 총계 포함
)
# 행 인덱스 정렬
tier_counts = tier_counts.reindex(target_groups + ['All'])
print(tier_counts)
print("\n" + "="*50)

print("### 📈 그룹별 사료 등급 선호도 (비율) ###")

# 2. 비율표 (Percentage by Group - 행 합계 100%)
# normalize='index'를 사용하여 각 그룹 내에서의 비율을 계산
tier_proportions = pd.crosstab(
    df_filtered['Final_Group'], 
    df_filtered['pet_food_tier'], 
    normalize='index' # 행별 비율 계산 (각 그룹 내에서 100%)
) * 100

# 행 인덱스 정렬 및 소수점 2자리로 포맷
tier_proportions = tier_proportions.reindex(target_groups)
tier_proportions = tier_proportions.applymap(lambda x: f'{x:.2f}%')

print(tier_proportions)
print("-" * 50)

### 📊 그룹별 사료 등급 선호도 (빈도) ###
pet_food_tier   mid  premium  superpremium    All
Final_Group                                      
A. 노령견 그룹      4939     5270         11770  21979
B. 알레르기 그룹      850      301          2453   3604
D. 일반 그룹       4056     2817          9432  16305
All            9845     8388         23655  41888

### 📈 그룹별 사료 등급 선호도 (비율) ###
pet_food_tier     mid premium superpremium
Final_Group                               
A. 노령견 그룹      22.47%  23.98%       53.55%
B. 알레르기 그룹     23.58%   8.35%       68.06%
D. 일반 그룹       24.88%  17.28%       57.85%
--------------------------------------------------


C:\Users\user\AppData\Local\Temp\ipykernel_14820\988320493.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  tier_proportions = tier_proportions.applymap(lambda x: f'{x:.2f}%')


다른거

In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import numpy as np

# 1. 데이터 로드 및 20,000개 랜덤 샘플 추출
df = pd.read_csv('df_dropped_dry_data.csv')

# 💡 20,000개 랜덤 샘플 추출
n_samples = min(20000, len(df))
df_sampled = df.sample(n=n_samples, random_state=42).copy()

# 2. 종속 변수 및 독립 변수 설정
target_col = 'total_order_kcal' # 종속 변수 (칼로리 섭취량)
feature_cols = [
    'pet_gender_pre',         # 성별
    'neutered_cleaned',       # 중성화 여부
    'pet_life_pre',           # 생애 주기
    'pet_breed_size_pre',     # 크기 (추가했으나, 첨부 이미지에서는 제외됨. 이미지와 일치시키려면 제외 가능)
    'kibble_kcal'             # 사료 칼로리
] 
# Note: 첨부 이미지에는 pet_breed_size_pre가 제외되어 있습니다.
# 만약 이미지와 동일하게 하려면 feature_cols를 조정해야 합니다.
# 여기서는 요청하신 변수(성별, 중성화, 생애주기, 크기)를 포함합니다.

df_reg = df_sampled[[target_col] + feature_cols].copy()

# 3. 전처리
# 종속 변수 결측치 제거
df_reg.dropna(subset=[target_col], inplace=True) 

# 범주형 변수 정의 및 결측치 'Unknown' 처리
categorical_cols = [
    'pet_gender_pre', 'neutered_cleaned', 'pet_life_pre', 
    'pet_breed_size_pre'
]
for col in categorical_cols:
    if col in df_reg.columns:
        df_reg[col] = df_reg[col].astype(str).replace('nan', 'Unknown').fillna('Unknown')

# One-Hot Encoding 적용
X_encoded = pd.get_dummies(df_reg[feature_cols], drop_first=True)
y = df_reg[target_col]

# 4. statsmodels OLS 모델 구축 및 학습
# statsmodels는 상수항(const)을 자동으로 추가하지 않으므로 명시적으로 추가해야 합니다.
X = sm.add_constant(X_encoded, prepend=False) 

# OLS 모델 적합 (학습)
model = sm.OLS(y, X)
results = model.fit()

# 5. OLS 결과 출력 (요청하신 이미지 형식)
print(results.summary())

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import numpy as np

# 1. 데이터 로드 및 20,000개 랜덤 샘플 추출
df = pd.read_csv('df_dropped_dry_data.csv')

# 💡 20,000개 랜덤 샘플 추출
n_samples = min(30000, len(df))
df_sampled = df.sample(n=n_samples, random_state=27).copy()

# 2. 종속 변수 및 독립 변수 설정
target_col = 'total_order_kcal' # 종속 변수 (칼로리 섭취량)
feature_cols = [
    'pet_gender_pre',         # 성별
    'pet_neutered_pre',       # 중성화 여부
    'pet_life_pre',           # 생애 주기
    'pet_breed_size_pre'     # 크기
               
] 

df_reg = df_sampled[[target_col] + feature_cols].copy()

# 3. 전처리
# 종속 변수 결측치 제거
df_reg.dropna(subset=[target_col], inplace=True) 

# **오류 해결을 위한 중요 단계:** 종속 변수를 실수형으로 강제 변환 (문자열 방지)
df_reg[target_col] = pd.to_numeric(df_reg[target_col], errors='coerce').fillna(0) # 변환 불가 시 0으로 채움

# 범주형 변수 정의 및 결측치 'Unknown' 처리
categorical_cols = [
    'pet_gender_pre', 'pet_neutered_pre', 'pet_life_pre', 
    'pet_breed_size_pre'
]
for col in categorical_cols:
    if col in df_reg.columns:
        df_reg[col] = df_reg[col].astype(str).replace('nan', 'Unknown').fillna('Unknown')

# One-Hot Encoding 적용
X_encoded = pd.get_dummies(df_reg[feature_cols], drop_first=True)
y = df_reg[target_col]

# **오류 해결을 위한 중요 단계:** 독립 변수 X_encoded를 실수형으로 강제 변환
X_encoded = X_encoded.astype(float) 

# 4. statsmodels OLS 모델 구축 및 학습
X = sm.add_constant(X_encoded, prepend=False) 

print(f"총 샘플 수: {len(df_reg)}개")
print("-" * 40)

# OLS 모델 적합 (학습)
model = sm.OLS(y, X)
results = model.fit()

# 5. OLS 결과 출력 (요청하신 이미지 형식)
print(results.summary())

총 샘플 수: 30000개
----------------------------------------
                            OLS Regression Results                            
Dep. Variable:       total_order_kcal   R-squared:                       0.582
Model:                            OLS   Adj. R-squared:                  0.581
Method:                 Least Squares   F-statistic:                     4631.
Date:                Thu, 13 Nov 2025   Prob (F-statistic):               0.00
Time:                        13:51:36   Log-Likelihood:            -3.1408e+05
No. Observations:               30000   AIC:                         6.282e+05
Df Residuals:                   29990   BIC:                         6.283e+05
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import numpy as np

# 1. 데이터 로드 및 30,000개 랜덤 샘플 추출 (동일 유지)
df = pd.read_csv('df_dropped_dry_data.csv')
n_samples = min(30000, len(df))
df_sampled = df.sample(n=n_samples, random_state=27).copy()

# 2. 종속 변수 및 독립 변수 설정 (동일 유지)
target_col = 'total_order_kcal' 
feature_cols = [
    'pet_gender_pre', 'pet_neutered_pre', 'pet_life_pre', 'pet_breed_size_pre'
] 
df_reg = df_sampled[[target_col] + feature_cols].copy()

# 3. 전처리 (동일 유지)
df_reg.dropna(subset=[target_col], inplace=True) 
df_reg[target_col] = pd.to_numeric(df_reg[target_col], errors='coerce').fillna(0) 
categorical_cols = ['pet_gender_pre', 'pet_neutered_pre', 'pet_life_pre', 'pet_breed_size_pre']
for col in categorical_cols:
    if col in df_reg.columns:
        df_reg[col] = df_reg[col].astype(str).replace('nan', 'Unknown').fillna('Unknown')

# 🚨 수정된 핵심 부분:
# 1) drop_first=False로 모든 더미 변수 생성
X_encoded = pd.get_dummies(df_reg[feature_cols], drop_first=False)
y = df_reg[target_col]

# 2) 수동으로 'pet_gender_pre_1' 제거 (기준 그룹 설정)
# 이렇게 하면 'pet_gender_pre_0'의 계수가 유효해집니다.
if 'pet_gender_pre_1' in X_encoded.columns:
    X_encoded.drop(columns=['pet_gender_pre_1'], inplace=True)

# 다중공선성을 해결하기 위해 다른 범주형 변수도 하나씩 제거해야 합니다.
# (이전에 drop_first=True로 처리했던 것과 동일한 효과를 내기 위해)
# 만약 'pet_neutered_pre_1'이 기준이 아니라 'pet_neutered_pre_0'을 보고 싶다면,
# 해당 변수도 제거해야 하지만, 여기서는 젠더만 요청하셨으므로 다른 변수는
# 기존 'drop_first=True' 효과를 내는 변수를 제거하겠습니다.
# (예시: pet_neutered_pre_0, pet_life_pre_0 등)

# 만약 'pet_neutered_pre_1'과 'pet_life_pre_3', 'pet_breed_size_pre_4'가 기준 그룹이 되길 원한다면,
# 해당 변수들을 제거해야 합니다. (이전 결과의 변수명을 기준으로 임의로 제거하겠습니다.)
# 현재의 범주가 0, 1, 2, 3, 4로 되어있다고 가정하고 가장 높은 값의 더미 변수들을 제거하여 기준을 설정합니다.
variables_to_drop = [
    'pet_neutered_pre_1', 
    'pet_life_pre_3', 
    'pet_breed_size_pre_4'
]

for col in variables_to_drop:
    if col in X_encoded.columns:
        X_encoded.drop(columns=[col], inplace=True)


X_encoded = X_encoded.astype(float) 

# 4. statsmodels OLS 모델 구축 및 학습
X = sm.add_constant(X_encoded, prepend=False) 

print(f"총 샘플 수: {len(df_reg)}개")
print("-" * 40)

# OLS 모델 적합 (학습)
model = sm.OLS(y, X)
results = model.fit()

# 5. OLS 결과 출력
print(results.summary())

총 샘플 수: 30000개
----------------------------------------
                            OLS Regression Results                            
Dep. Variable:       total_order_kcal   R-squared:                       0.582
Model:                            OLS   Adj. R-squared:                  0.581
Method:                 Least Squares   F-statistic:                     4631.
Date:                Thu, 13 Nov 2025   Prob (F-statistic):               0.00
Time:                        15:23:21   Log-Likelihood:            -3.1408e+05
No. Observations:               30000   AIC:                         6.282e+05
Df Residuals:                   29990   BIC:                         6.283e+05
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import numpy as np

# 1. 데이터 로드 및 30,000개 랜덤 샘플 추출
df = pd.read_csv('df_dropped_dry_data.csv')
n_samples = min(30000, len(df))
df_sampled = df.sample(n=n_samples, random_state=27).copy()

# 2. 종속 변수 및 독립 변수 설정
target_col = 'total_order_kcal' 
feature_cols = [
    'pet_gender_pre', 'pet_neutered_pre', 'pet_life_pre', 'pet_breed_size_pre'
] 
df_reg = df_sampled[[target_col] + feature_cols].copy()

# 3. 전처리
df_reg.dropna(subset=[target_col], inplace=True) 
df_reg[target_col] = pd.to_numeric(df_reg[target_col], errors='coerce').fillna(0) 
categorical_cols = ['pet_gender_pre', 'pet_neutered_pre', 'pet_life_pre', 'pet_breed_size_pre']
for col in categorical_cols:
    if col in df_reg.columns:
        df_reg[col] = df_reg[col].astype(str).replace('nan', 'Unknown').fillna('Unknown')

# 🚨 핵심 수정 부분: One-Hot Encoding 및 기준 그룹 설정
# 1) drop_first=False로 모든 더미 변수 생성
X_encoded = pd.get_dummies(df_reg[feature_cols], drop_first=False)
y = df_reg[target_col]

# 2) 요청하신 기준 변수들을 수동으로 제거 (기준 그룹 설정)
# 제거할 변수 목록
reference_vars_to_drop = [
    'pet_gender_pre_1',       # 기준 1: 젠더 1
    'pet_neutered_pre_1',     # 기준 2: 중성화 1
    'pet_life_pre_3',         # 기준 3: 펫 생애주기 3
    'pet_breed_size_pre_0'    # 기준 4: 펫 브리드 사이즈 0
]

for col in reference_vars_to_drop:
    if col in X_encoded.columns:
        X_encoded.drop(columns=[col], inplace=True)

# 데이터 타입 정리
X_encoded = X_encoded.astype(float) 

# 4. statsmodels OLS 모델 구축 및 학습
X = sm.add_constant(X_encoded, prepend=False) 

print(f"총 샘플 수: {len(df_reg)}개")
print("-" * 40)

# OLS 모델 적합 (학습)
model = sm.OLS(y, X)
results = model.fit()

# 5. OLS 결과 출력
print(results.summary())

총 샘플 수: 30000개
----------------------------------------
                            OLS Regression Results                            
Dep. Variable:       total_order_kcal   R-squared:                       0.582
Model:                            OLS   Adj. R-squared:                  0.581
Method:                 Least Squares   F-statistic:                     4631.
Date:                Thu, 13 Nov 2025   Prob (F-statistic):               0.00
Time:                        15:27:03   Log-Likelihood:            -3.1408e+05
No. Observations:               30000   AIC:                         6.282e+05
Df Residuals:                   29990   BIC:                         6.283e+05
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [35]:
df['pet_neutered_pre']

0        1
1        1
2        1
3        1
4        1
        ..
34108    1
34109    1
34110    1
34111    0
34112    0
Name: pet_neutered_pre, Length: 34113, dtype: int64